# Notebook para manutenção do código

In [60]:
import seaborn as sns

## local_lib

In [61]:
#@st.cache
def etl_tesouro_historic_price():
    url = 'https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv'
    df = pd.read_csv(url, sep=';', decimal=',')
    df['data'] = pd.to_datetime(df['Data Base'], format='%d/%m/%Y')
    df['ticker'] = df['Tipo Titulo'].astype(str) + ' ' + df['Data Vencimento'].str[6:]
    df['preco_hist'] = round(df['PU Base Manha'].astype(float), 2)
    return df

#-----------------------------------------------------------------------------------------------------
#@st.cache  
def etl_benchmark_historic_price():
    # CDI
    df_cdi = pd.read_json('http://api.bcb.gov.br/dados/serie/bcdata.sgs.12/dados?formato=json')
    df_cdi['data'] = pd.to_datetime(df_cdi['data'], format='%d/%m/%Y')
    df_cdi.columns = ['data','cdi']

    # IPCA
    df_ipca = pd.read_json('http://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?formato=json')
    df_ipca['data'] = pd.to_datetime(df_ipca['data'], format='%d/%m/%Y')
    df_ipca.columns = ['data','ipca']
    df_ipca['ipca'] = round((1 + df_ipca['ipca']) ** (1/22) - 1, 6)

    # IBOV
    df_ibov = yf.download('^BVSP', interval='1d')['Adj Close'].reset_index(drop=False)
    df_ibov.columns = ['data','ibov']
    df_ibov['ibov'] = ((df_ibov['ibov'] / df_ibov['ibov'].shift(1) - 1) * 100).fillna(0).round(6)
        
    # S&P500
    df_sp500 = yf.download('^GSPC', interval='1d')['Adj Close'].reset_index(drop=False)
    df_sp500.columns = ['data','sp500']
    df_sp500['sp500'] = ((df_sp500['sp500'] / df_sp500['sp500'].shift(1) - 1) * 100).fillna(5).round(6)

    df_final = pd.merge(df_ibov, df_sp500, on='data', how='inner')
    df_final = pd.merge(df_final, df_cdi, on='data', how='inner')
    df_final = pd.merge(df_final, df_ipca, on='data', how='left')
    df_final['ipca'] = df_final['ipca'].fillna(method='ffill')
    df_final['data'] = pd.to_datetime(df_final['data'])

    return df_final

## Tesouro Direto

In [62]:
import pandas as pd
import yfinance as yf
import local_lib as lib

### Parte 1: Tabela Dinâmica

In [63]:
# Carregando os dados inputados pelo usuário.
df_tesouro = pd.read_csv('../data/manutencao/dados_pos_home.csv')
df_tesouro['data'] = pd.to_datetime(df_tesouro['data'], format='%Y-%m-%d')
df_tesouro.head()

,data,ticker,qt,preco_mov,vl_total
0,2021-06-30,Tesouro IPCA+ 2035,1.70,2045.17,3476.79
1,2022-07-01,Tesouro IPCA+ 2035,0.00,0.00,-3.14
2,2021-11-04,Tesouro IPCA+ 2045,3.30,1069.94,3530.80
3,2022-07-01,Tesouro IPCA+ 2045,0.00,0.00,-3.48
4,2020-07-10,Tesouro Prefixado 2023,1.63,903.07,1472.00


In [64]:
# Carregando os preços diários extraídos da API do Tesouro.
df_hist_tesouro = etl_tesouro_historic_price()
df_hist_tesouro.head()

,Tipo Titulo,Data Vencimento,Data Base,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha,data,ticker,preco_hist
0,Tesouro IPCA+ com Juros Semestrais,15/08/2024,09/10/2012,3.70,3.78,2676.99,2658.66,2657.76,2012-10-09,Tesouro IPCA+ com Juros Semestrais 2024,2657.76
1,Tesouro Prefixado,01/01/2013,05/07/2012,7.63,7.66,965.02,964.89,964.61,2012-07-05,Tesouro Prefixado 2013,964.61
2,Tesouro Prefixado,01/01/2014,06/08/2012,7.84,7.88,899.66,899.20,898.92,2012-08-06,Tesouro Prefixado 2014,898.92
3,Tesouro Prefixado,01/01/2015,06/08/2012,8.36,8.42,824.41,823.32,823.05,2012-08-06,Tesouro Prefixado 2015,823.05
4,Tesouro Prefixado com Juros Semestrais,01/01/2014,06/08/2012,7.84,7.88,1036.60,1036.09,1035.77,2012-08-06,Tesouro Prefixado com Juros Semestrais 2014,1035.77


In [65]:
# Juntando os dados do usuário com os dados da API.
df_tesouro_historico = lib.merge_historic_tesouro(df_hist_tesouro, df_tesouro)
df_tesouro_historico.head()

,Tipo Titulo,Data Vencimento,Data Base,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha,data,ticker,preco_hist,qt,preco_mov,vl_total,qt_acum,vl_atualizado
300,Tesouro IPCA+,15/05/2035,30/06/2021,4.14,4.26,2035.06,2002.91,2002.19,2021-06-30,Tesouro IPCA+ 2035,2002.19,1.7,2045.17,3476.79,1.7,3403.72
301,Tesouro IPCA+,15/05/2035,01/07/2021,4.15,4.27,2033.07,2000.97,2000.26,2021-07-01,Tesouro IPCA+ 2035,2000.26,0.0,0.00,0.00,1.7,3400.44
302,Tesouro IPCA+,15/05/2035,02/07/2021,4.13,4.25,2039.99,2007.78,2006.29,2021-07-02,Tesouro IPCA+ 2035,2006.29,0.0,0.00,0.00,1.7,3410.69
303,Tesouro IPCA+,15/05/2035,05/07/2021,4.11,4.23,2046.13,2013.83,2013.11,2021-07-05,Tesouro IPCA+ 2035,2013.11,0.0,0.00,0.00,1.7,3422.29
304,Tesouro IPCA+,15/05/2035,06/07/2021,4.12,4.24,2044.14,2011.88,2011.16,2021-07-06,Tesouro IPCA+ 2035,2011.16,0.0,0.00,0.00,1.7,3418.97


In [66]:
# Passo extra para criar uma coluna identificando o último dia do mês de cada ativo (útil para os plots)
df_tesouro_historico = lib.create_column_last_day(df_tesouro_historico)
df_tesouro_historico.head()

c:\Users\Kenji\GitHub\b3-report\src\local_lib.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right['year'] = pd.to_datetime(df_right['data']).dt.year
c:\Users\Kenji\GitHub\b3-report\src\local_lib.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right['month'] = pd.to_datetime(df_right['data']).dt.month
c:\Users\Kenji\GitHub\b3-report\src\local_lib.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

,Tipo Titulo,Data Vencimento,Data Base,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha,data,ticker,preco_hist,qt,preco_mov,vl_total,qt_acum,vl_atualizado,dummy_ultimo_dia,dt_competencia
0,Tesouro IPCA+,15/05/2035,30/06/2021,4.14,4.26,2035.06,2002.91,2002.19,2021-06-30,Tesouro IPCA+ 2035,2002.19,1.7,2045.17,3476.79,1.7,3403.72,1.0,2021-06-01
1,Tesouro IPCA+,15/05/2035,01/07/2021,4.15,4.27,2033.07,2000.97,2000.26,2021-07-01,Tesouro IPCA+ 2035,2000.26,0.0,0.00,0.00,1.7,3400.44,0.0,2021-07-01
2,Tesouro IPCA+,15/05/2035,02/07/2021,4.13,4.25,2039.99,2007.78,2006.29,2021-07-02,Tesouro IPCA+ 2035,2006.29,0.0,0.00,0.00,1.7,3410.69,0.0,2021-07-01
3,Tesouro IPCA+,15/05/2035,05/07/2021,4.11,4.23,2046.13,2013.83,2013.11,2021-07-05,Tesouro IPCA+ 2035,2013.11,0.0,0.00,0.00,1.7,3422.29,0.0,2021-07-01
4,Tesouro IPCA+,15/05/2035,06/07/2021,4.12,4.24,2044.14,2011.88,2011.16,2021-07-06,Tesouro IPCA+ 2035,2011.16,0.0,0.00,0.00,1.7,3418.97,0.0,2021-07-01


In [67]:
# Selecionar os tickers a serem visualizados (desnecessário neste notebook)
'''
# Filtro de tickers.
list_ticker = st.multiselect('Escolha o(s) investimento(s):',
                                df_tesouro_historico['ticker'].unique().tolist(),
                                df_tesouro_historico['ticker'].unique().tolist())
df_tesouro_historico = df_tesouro_historico.loc[df_tesouro_historico['ticker'].isin(list_ticker)]
'''

"\n# Filtro de tickers.\nlist_ticker = st.multiselect('Escolha o(s) investimento(s):',\n                                df_tesouro_historico['ticker'].unique().tolist(),\n                                df_tesouro_historico['ticker'].unique().tolist())\ndf_tesouro_historico = df_tesouro_historico.loc[df_tesouro_historico['ticker'].isin(list_ticker)]\n"

In [68]:
# Preparar os dados exclusivamente para a Tabela Dinâmica (visão 1)
df_plot = df_tesouro_historico.loc[df_tesouro_historico['dummy_ultimo_dia'] == 1]
tab1, data_col = lib.custom_pivot_table(df_plot, col_value='vl_atualizado')
tab1.head()

c:\Users\Kenji\GitHub\b3-report\src\local_lib.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dt_competencia'] = pd.to_datetime(df['dt_competencia']).dt.date


,ticker,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,...,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,Total
0,Tesouro IPCA+ 2035,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,3232.96,3231.14,3204.99,3095.68,3208.58,3286.29,3257.90,3240.27,3124.67,60945.93
1,Tesouro IPCA+ 2045,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,3597.03,3554.60,3476.35,3255.32,3470.44,3618.65,3502.26,3445.20,3185.36,49075.29
2,Tesouro Prefixado 2023,0.0,0.0,0.0,1487.99,1478.72,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2966.71
3,Tesouro Prefixado 2026,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,3285.75,3272.25,3262.70,3290.00,3408.65,3470.55,3488.75,3410.90,3374.35,63186.10
4,Tesouro Selic 2024,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,7052.97,7131.71,7207.41,7283.94,7372.73,7455.93,7534.35,7610.52,7665.60,155841.05


### Parte 2: Gráfico de Linha com Benchmark

In [69]:
# Extração das variações dos benchmarks (API do BC e yfinance)
df_hist_bench = etl_benchmark_historic_price()
df_hist_bench.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,data,ibov,sp500,cdi,ipca
0,1993-04-27,0.000000,1.031047,1.389667,NaN
1,1993-04-28,-0.816330,0.002278,1.374333,NaN
2,1993-04-29,-2.469130,0.198627,1.346333,NaN
3,1993-04-30,1.687762,0.296199,1.302333,NaN
4,1993-05-03,0.000000,0.515684,1.305333,NaN


In [70]:
# Junção dos dados inputados pelo usuário e os dados históricos do benchmarks
df_tesouro_historico_agg = lib.merge_historic_benchmark(df_tesouro_historico, df_hist_bench)
df_tesouro_historico_agg.head()

,data,ibov,sp500,cdi,ipca,vl_total,vl_atualizado
0,2020-04-14,1.372469,3.057259,0.014227,-0.016725,2960.86,2959.40
1,2020-04-15,-1.360144,-2.203044,0.014227,-0.016725,0.00,2959.83
2,2020-04-16,-1.292639,0.581669,0.014227,-0.016725,2221.28,5180.45
3,2020-04-17,1.513905,2.679359,0.014227,-0.016725,0.00,5181.19
4,2020-04-20,-0.021522,-1.788105,0.014227,-0.016725,0.00,5181.94


In [71]:
# Arrumando dados em um formato melhor para o gráfico.
tab2 = lib.custom_data_lineplot(df_tesouro_historico_agg, ['ibov', 'sp500', 'cdi', 'ipca'])
tab2.head()

,data,variable,value
0,2020-04-14,Carteira,2959.40
1,2020-04-15,Carteira,2959.83
2,2020-04-16,Carteira,5180.45
3,2020-04-17,Carteira,5181.19
4,2020-04-20,Carteira,5181.94


### Parte 3: KPI

In [72]:
df_tesouro_historico.head()

,Tipo Titulo,Data Vencimento,Data Base,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha,data,ticker,preco_hist,qt,preco_mov,vl_total,qt_acum,vl_atualizado,dummy_ultimo_dia,dt_competencia
0,Tesouro IPCA+,15/05/2035,30/06/2021,4.14,4.26,2035.06,2002.91,2002.19,2021-06-30,Tesouro IPCA+ 2035,2002.19,1.7,2045.17,3476.79,1.7,3403.72,1.0,2021-06-01
1,Tesouro IPCA+,15/05/2035,01/07/2021,4.15,4.27,2033.07,2000.97,2000.26,2021-07-01,Tesouro IPCA+ 2035,2000.26,0.0,0.00,0.00,1.7,3400.44,0.0,2021-07-01
2,Tesouro IPCA+,15/05/2035,02/07/2021,4.13,4.25,2039.99,2007.78,2006.29,2021-07-02,Tesouro IPCA+ 2035,2006.29,0.0,0.00,0.00,1.7,3410.69,0.0,2021-07-01
3,Tesouro IPCA+,15/05/2035,05/07/2021,4.11,4.23,2046.13,2013.83,2013.11,2021-07-05,Tesouro IPCA+ 2035,2013.11,0.0,0.00,0.00,1.7,3422.29,0.0,2021-07-01
4,Tesouro IPCA+,15/05/2035,06/07/2021,4.12,4.24,2044.14,2011.88,2011.16,2021-07-06,Tesouro IPCA+ 2035,2011.16,0.0,0.00,0.00,1.7,3418.97,0.0,2021-07-01


In [83]:
vl_aporte = df_tesouro_historico.loc[(df_tesouro_historico['qt'] != 0) & (df_tesouro_historico['vl_total'] > 0), 'vl_total'].sum()
vl_resgate = df_tesouro_historico.loc[(df_tesouro_historico['qt'] != 0) & (df_tesouro_historico['vl_total'] < 0), 'vl_total'].sum() * -1
vl_investido = df_tesouro_historico.loc[df_tesouro_historico['data']==df_tesouro_historico['data'].max(), 'vl_atualizado'].sum()

rendimento_nominal = round((vl_investido + vl_resgate - vl_aporte) / vl_aporte * 100 , 2)

vl_aporte, vl_resgate, vl_investido, rendimento_nominal


(86689.84, 33237.02, 58900.16, 6.28)

In [96]:
df_tesouro_historico['data'].max().to_pydatetime()

datetime.datetime(2022, 12, 20, 0, 0)

In [97]:
df_tesouro_historico.loc[df_tesouro_historico['data'] == df_tesouro_historico['data'].max().to_pydatetime()]

,Tipo Titulo,Data Vencimento,Data Base,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha,data,ticker,preco_hist,qt,preco_mov,vl_total,qt_acum,vl_atualizado,dummy_ultimo_dia,dt_competencia
368,Tesouro IPCA+,15/05/2035,20/12/2022,6.35,6.47,1864.62,1838.04,1838.04,2022-12-20,Tesouro IPCA+ 2035,1838.04,0.0,0.0,0.0,1.70,3124.67,1.0,2022-12-01
650,Tesouro IPCA+,15/05/2045,20/12/2022,6.44,6.56,990.27,965.26,965.26,2022-12-20,Tesouro IPCA+ 2045,965.26,0.0,0.0,0.0,3.30,3185.36,1.0,2022-12-01
1077,Tesouro Prefixado,01/01/2026,20/12/2022,13.71,13.83,677.38,674.87,674.87,2022-12-20,Tesouro Prefixado 2026,674.87,0.0,0.0,0.0,5.00,3374.35,1.0,2022-12-01
1431,Tesouro Selic,01/09/2024,20/12/2022,0.01,0.02,12575.11,12566.55,12566.55,2022-12-20,Tesouro Selic 2024,12566.55,0.0,0.0,0.0,0.61,7665.60,1.0,2022-12-01
2100,Tesouro Selic,01/03/2025,20/12/2022,0.03,0.04,12569.33,12560.12,12560.12,2022-12-20,Tesouro Selic 2025,12560.12,0.0,0.0,0.0,1.49,18714.58,1.0,2022-12-01
2301,Tesouro Selic,01/03/2027,20/12/2022,0.17,0.18,12490.15,12478.47,12478.47,2022-12-20,Tesouro Selic 2027,12478.47,0.0,0.0,0.0,1.83,22835.60,1.0,2022-12-01


In [86]:
import datetime 

In [87]:
datetime.datetime(2022, 12, 20)

datetime.datetime(2022, 12, 20, 0, 0)